# Homework

1. Complete the Python implementation of the backpropagation exercise in the **Backpropagation** section here above (cell `# try it in Python as homework!`)
    - Create the calculations for obtaining $y$ in PyTorch **using only PyTorch methods and routines**
    - Calculate the gradient
    - Check the values of the gradients and see if it is correct w.r.t. the manual calculations
2. Given the multilayer perceptron defined during the exercises from lab 1:
    - Create 10 random datapoints (with any function you wish, it can be `rand`, `randn`...) and feed them into the network
    - Given the output, calculate the Cross-Entropy loss with respect to the ground truth $[1,2,3,4,1,2,3,4,1,2]$ (classes from 1 to 4). Cross-Entropy loss:
        
        $$ CE(\mathbf{y}, \hat{\mathbf{y}}) = - \sum_{i=1}^{10} \hat{y}_i \log(y_i)$$
        
        where $y_i$ is the one-hot encoding of the $i$-th datapoint. For instance, $y_1 = [1,0,0,0]$.
        **_Note: there is an extremely handy PyTorch function for getting a one-hot encoding out of a vector, so don't try anything fancy._**
    - Backpropagate the error along the network and inspect the gradient of the parameters connecting the input layer and the first hidden layer.
3. Execute the python script `utils/randomized_backpropagation_formula.py`. This creates a formula $f(\mathbf{x})$ with randomized operators and values. Create the computational graph from this formula, do (by hand) the forward pass, then calculate (by hand) $\nabla f(\mathbf{x})$ using the backward gradient computation. Do the same calculation on PyTorch to check the correctness of your calculations. _Note: The formula created by this script is linked to your name and surname, which you have to input before_. The solution to this exercise _should_ be submitted as a scan/good quality picture of a piece of paper (or you can do it on a touch screen and submit the image...), but other formats are acceptable as well.

In [26]:
import torch

## Exercise 1

**Backpropagation**

Let us suppose we have the following calculation

$\mathbf{x} = [1,~2,~-1,~3,~5]$

$ y = f(\mathbf{x}) = \log\{[\exp (x_1 * x_2 )]^2 + \sin (x_3 + x_4 + x_5) \cdot x_5\}$

Find

$\nabla f(\mathbf{x})$

### Part 1

In [27]:
x1 = torch.tensor([1.0], requires_grad=True)
x2 = torch.tensor([2.0], requires_grad=True)
x3 = torch.tensor([-1.0], requires_grad=True)
x4 = torch.tensor([3.0], requires_grad=True)
x5 = torch.tensor([5.0], requires_grad=True)

print(x1)
print(x2)
print(x3)
print(x4)
print(x5)

# x = torch.tensor([1,2,-1,3,5], dtype=torch.float32, requires_grad=True)
# print(x)

tensor([1.], requires_grad=True)
tensor([2.], requires_grad=True)
tensor([-1.], requires_grad=True)
tensor([3.], requires_grad=True)
tensor([5.], requires_grad=True)


In [30]:
a = torch.matmul(x1, x2)
print(a)
b = x3 + x4 + x5
print(b)
c = a.exp()
print(c)
d = torch.pow(c, 2)
print(d)
g = b.sin()
print(g)
h = torch.matmul(g, x5)
print(h)
i = d + h
print(i)
y = torch.log(i)
print("\n")
print("The value of function f(x) evaluated in x is:")
print(y)

tensor(2., grad_fn=<DotBackward0>)
tensor([7.], grad_fn=<AddBackward0>)
tensor(7.3891, grad_fn=<ExpBackward0>)
tensor(54.5982, grad_fn=<PowBackward0>)
tensor([0.6570], grad_fn=<SinBackward0>)
tensor(3.2849, grad_fn=<DotBackward0>)
tensor(57.8831, grad_fn=<AddBackward0>)


The value of function f(x) evalkuated in x is:
tensor(4.0584, grad_fn=<LogBackward0>)


### Part 2

In [31]:
y.backward()
print(x1.grad)
print(x2.grad)
print(x3.grad)
print(x4.grad)
print(x5.grad)

tensor([3.7730])
tensor([1.8865])
tensor([0.0651])
tensor([0.0651])
tensor([0.0765])


### Part 3

From manual calculations performed in class, we expected:

- $\partial f/\partial x_1 = 4.14$
- $\partial f/\partial x_2 = 2.07$
- $\partial f/\partial x_3 = 0.08$
- $\partial f/\partial x_4 = 0.08$
- $\partial f/\partial x_4 = 0.093$

As you can see from previous cell in Part 2, the manual calculations are slightly different since we made use of approximations in class.

## Exercise 3

Input your name, then press ENTER: Valeria Insogna

f(X) =  exp((sin(x1 + x2) / ReLU(x3 + x4)) - x5)

Your values
{'x1': 3, 'x2': 4, 'x3': -2, 'x4': -1, 'x5': -2}

Calculate ∇f(X) [NB: if division by 0, change the value(s) of X responsible for that]

In [ ]:
x_1 = torch.tensor([3.0], requires_grad=True)
x_2 = torch.tensor([4.0], requires_grad=True)
x_3 = torch.tensor([2.0], requires_grad=True) #I have change it from -2 to +2 otherwise there is a  division with denominator 0! 
x_4 = torch.tensor([-1.0], requires_grad=True)
x_5 = torch.tensor([-2.0], requires_grad=True)

print(x_1)
print(x_2)
print(x_3)
print(x_4)
print(x_5)

Let's first evaluate the forward function step by step:

In [ ]:
a = x_1 + x_2
print(a)
b = x_3 + x_4
print(b)
c = a.sin()
print(c)
d = torch.nn.functional.relu(b)
print(d)
e = c/d
print(e)
f = e - x_5
print(f)
g = f.exp()
print(g)


This is the image of the manual calculations for the backward propagation:

![](ex2.png)

Now let's verify the manual counts:

In [ ]:
g.backward()
print(x_1.grad)
print(x_2.grad)
print(x_3.grad)
print(x_4.grad)
print(x_5.grad)